In [81]:
import numpy as np
N, T, dX, dU,dV = 10, 100,26, 7, 7

l = np.zeros(T)
lu = np.zeros((T, dU))
lv = np.zeros((T, dV))
lx = np.zeros((T, dX))
luu = np.zeros((T, dU, dU))
lvv = np.zeros((T, dU, dV))
lxx = np.zeros((T, dX, dX))
lux = np.zeros((T, dU, dX))
lvx = np.zeros((T, dV, dX))

# Compute cost.
cs = np.zeros((N, T))  # see algorithm_utils.py
cc = np.zeros((N, T))   # Cost estimate constant term.
cv = np.zeros((N, T, dX+dU+dV))    # Cost estimate vector term.
Cm = np.zeros((N, T, dX+dU+dV, dX+dU+dV)) # Cost estimate matrix term.

term1 = np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])]
term2 = np.c_[lux, luu, lvv]
cv_temp = np.c_[lx, lu,  lv]
term3 = np.c_[lvx, lvv]
Cm_ord = np.concatenate(
                (np.c_[lxx, np.transpose(lux, [0, 2, 1])], np.c_[lux, luu]),
                axis=1
            )
Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )   
Cm2 = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)
# # print('term1: {}, Cm: {}, term2: {}, term3: {}'.format(term1.shape, Cm.shape, term2.shape, term3.shape))
print('Cm : {} temp'.format(Cm_temp.shape), np.c_[lxx, np.transpose(lux, [0, 2, 1])].shape)
print(np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])].shape)
print(np.c_[lux, luu, lvv].shape)
print(np.c_[lvx, lvv, luu].shape)
print('Cm temp: ', Cm_temp.shape)
print('Cm2 shape', Cm2.shape)
for n in range(N):
    cc[n, :] = l
    cs[n, :] = l
    cv[n,:,:] = np.c_[lx, lu,  lv]
    Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )     
    Cm[n, :, :, :] = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)

print('cs: {}, cv: {}, Cm: {}'.format(cs.shape, cv.shape, Cm.shape))

('Cm : (100, 33, 40) temp', (100, 26, 33))
(100, 26, 40)
(100, 7, 40)
(100, 7, 40)
('Cm temp: ', (100, 33, 40))
('Cm2 shape', (100, 40, 40))
cs: (10, 100), cv: (10, 100, 40), Cm: (10, 100, 40, 40)


In [71]:
import numpy as np

dX, dU, dV = 26, 7, 7

Ltt = np.diag(np.hstack([
     np.ones(dU),
       np.ones(dU),
    np.zeros(dX - dU*2), np.ones(dU)
]))

Ltt2 = np.diag(np.hstack([
     np.ones(dU),
     np.ones(dU),
    np.zeros(dX - dU*2), np.ones(dU),
     np.ones(dV)])
              )
print('Ltt: ', Ltt.shape)
print('Ltt2: ', Ltt2.shape)
# x0 = np.concatenate([np.array([0.1, 0.1, -1.54, -1.7, 1.54, -0.2, 0]),
#                      np.zeros(7)])
# print(x0.shape, np.r_[x0, np.zeros(dU)].shape)
# lt = -Ltt.dot(np.r_[x0, np.zeros(dU)])

('Ltt: ', (33, 33))
('Ltt2: ', (40, 40))


In [72]:
import numpy as np
dX, dU, dV = 26, 7, 7
dt = 0.1
gains = 1.0/np.array([3.09, 1.08, 0.393, 0.674, 0.111, 0.152, 0.098])
Fd = np.vstack([
        np.hstack([
            np.eye(dU), dt * np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt ** 2 * np.diag(gains)
        ]),
        np.hstack([
            np.zeros((dU, dU)), np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt * np.diag(gains)
        ]),
        np.zeros((dX - dU*2, dX+dU))
    ])
acc = np.zeros(7)
fc = np.hstack([acc * dt ** 2, acc * dt, np.zeros((dX - dU*2))])

Fdr = np.vstack([
        np.hstack([
            np.eye(dU), dt * np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt ** 2 * np.diag(gains),
            np.eye(dV), #dt * np.eye(dV), np.zeros((dV, dX - dV*2)),
           # dt ** 2 * np.diag(gains),
        ]),
        np.hstack([
            np.zeros((dU, dU)), np.eye(dU), np.zeros((dU, dX - dU*2)),
            dt * np.diag(gains),
            np.zeros((dV, dV)), #np.eye(dV), np.zeros((dV, dX - dV*2)),
            #dt ** 2 * np.diag(gains),
        ]),
       np.zeros((dX - dU*2, dX+dU+dV))
    ])
fcr = np.hstack([acc * dt ** 2, acc * dt, np.zeros((dX - dU))])
print('fc: {}, Fd: {}, Fdr: {}, fcr: {}'.format(fc.shape, Fd.shape, Fdr.shape, fcr.shape))

fc: (26,), Fd: (26, 33), Fdr: (26, 40), fcr: (33,)


In [138]:
T, dU = 5, 7
O = np.ones((T, dU))
OO = 2*O
OOO = 3*O
Of = np.concatenate([O, OO, OOO], axis=0)

id0 = slice(15)
id1 = slice(15,dU)
id2 = slice(7)

Off = Of[id0, id0]
Qxu = Of[id1, id1]

s = np.random.normal(0, 1, Of.shape)
print(s)

[[ 0.21397828  0.439783   -0.22386141  1.22892886 -1.8709389   0.69199128
   0.81155184]
 [ 0.9663602  -1.01950591  0.49930228 -0.16615795 -0.1465245  -0.98963572
  -0.16551393]
 [ 1.25318746  0.56133317 -1.18017888  0.3638695  -0.41104636  0.92633215
   0.36225963]
 [-0.18157451  1.66601087 -0.29444132 -0.62786584 -0.78265027  0.01626098
  -1.77185853]
 [ 0.51189041 -0.21389767  1.83160416 -1.14229021  0.52727386 -1.66941454
  -0.45457236]
 [-0.56813289 -0.45827838 -1.29951204  0.2800272   1.07860613  0.12205078
   1.36941289]
 [ 0.44089889 -0.80487558  0.4124065  -0.9873763   0.61606522 -0.89238276
   1.35127981]
 [ 0.11277199 -1.07499321 -0.76748393 -0.02092747 -2.13453064 -1.53955451
  -1.08048905]
 [ 0.73435354  0.30253813 -0.10197845  0.52017667  0.17150618 -0.96575421
  -0.79418074]
 [ 0.26643123  1.79663389  1.11700885  0.90892892 -0.60262637 -0.56212744
   0.47194081]
 [-1.42166941 -0.06672178  0.38823906 -2.19519184  0.18290996 -1.13802641
   0.48272963]
 [ 1.00469654  1.1653

In [133]:
a = np.empty_like(Of)
help(np.random.normal)

Help on built-in function normal:

normal(...)
    normal(loc=0.0, scale=1.0, size=None)
    
    Draw random samples from a normal (Gaussian) distribution.
    
    The probability density function of the normal distribution, first
    derived by De Moivre and 200 years later by both Gauss and Laplace
    independently [2]_, is often called the bell curve because of
    its characteristic shape (see the example below).
    
    The normal distributions occurs often in nature.  For example, it
    describes the commonly occurring distribution of samples influenced
    by a large number of tiny, random disturbances, each with its own
    unique distribution [2]_.
    
    Parameters
    ----------
    loc : float or array_like of floats
        Mean ("centre") of the distribution.
    scale : float or array_like of floats
        Standard deviation (spread or "width") of the distribution.
    size : int or tuple of ints, optional
        Output shape.  If the given shape is, e.g., ``(m,